In [1]:
import utils_misc
utils_misc.select_freer_gpu()

from utils_qg_eval import InqQG_Eval, load_discord_qg_eval_dataset
from utils_qg_eval import compute_results
import tqdm, json

Will use GPU: 1


In [2]:
iqg_eval = InqQG_Eval()
dataset = load_discord_qg_eval_dataset()
print(len(dataset))

[2022-10-14 10:56:17,777] [WARNING] [partition_parameters.py:53:<module>] unable to find torch.distributed._all_gather_base. will fall back to torch.distributed.all_gather which will result in suboptimal performance. please consider upgrading your pytorch installation.


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-large and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to us

<All keys matched successfully>
Model loaded


Some weights of the model checkpoint at /export/home/models/quip-hf/ were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /export/home/models/quip-hf/ and are newly initialized: ['classifier.out_proj.bias', 'classifier.d

<All keys matched successfully>
100


In [3]:
from model_hf_generator import GeneratorHF

qgens = [
    # {"model_name": "bartl_aqg_2ep", "model": GeneratorHF(model_card="facebook/bart-large", starter_file="bartl_aqg_ep2_L_1.697.bin")},
    # {"model_name": "bartl_aqg_4ep", "model": GeneratorHF(model_card="facebook/bart-large", starter_file="/export/share/plaban/models/discordqg/bartl_dqg_L_1.509.bin")},
    # {"model_name": "bartl_1.43_fp16", "model": GeneratorHF(model_card="facebook/bart-large", starter_file="/export/home/models/bartl_dqg_bs32_L_1.433.bin")},
    # {"model_name": "bartl_1.442_fp16", "model": GeneratorHF(model_card="facebook/bart-large", starter_file="/export/home/models/bartl_dqg_bs32_L_1.442.bin")},
    # {"model_name": "bartl_1.456_fp16", "model": GeneratorHF(model_card="facebook/bart-large", starter_file="/export/home/models/bartl_dqg_bs32_L_1.456.bin")},
    {"model_name": "bartl_1.483_fp16", "model": GeneratorHF(model_card="facebook/bart-large", starter_file="/export/home/models/bartl_dqg_bs32_L_1.483.bin")},
    {"model_name": "bartl_1.504_fp16", "model": GeneratorHF(model_card="facebook/bart-large", starter_file="/export/home/models/bartl_dqg_bs32_L_1.504.bin")},
    # {"model_name": "mixqgl_aqg", "model": GeneratorHF(model_card="Salesforce/mixqg-large", starter_file="mixqgl_aqg_L_1.164.bin")},
]

for qgen in qgens:
    if "fp16" in qgen["model_name"]:
        qgen["model"].model.half()
print("Done loading models")

<All keys matched successfully>
<All keys matched successfully>
Done loading models


In [4]:
for start_word in ["How", "Why", "What", "Who"]:
    gen_questions = {qgen["model_name"]: [] for qgen in qgens}
    # Generate the Questions
    for d in tqdm.tqdm(dataset):
        for qgen in qgens:
            gen_qs = qgen["model"].generate([d["summary"]], max_gen_length=30, force_start=start_word)[0]
            gen_questions[qgen["model_name"]].append(gen_qs[0]["output_text"])

    # Score the questions
    iqg_scores = {qgen["model_name"]: [] for qgen in qgens}
    for i, context in enumerate(tqdm.tqdm(dataset)):
        for qgen in qgens:
            q = gen_questions[qgen["model_name"]][i]
            good_paragraphs = [p for a in context["articles"] for p in a["paragraphs"]]
            other_paragraphs = context["other_paragraphs"]
            iqg_score = iqg_eval.score_one(q, good_paragraphs, other_paragraphs)
            iqg_scores[qgen["model_name"]].append(iqg_score)
    print(compute_results(iqg_scores))

  0%|          | 0/100 [00:00<?, ?it/s]/export/home/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/export/home/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 100/100 [15:12<00:00,  9.12s/it]


[{'QG Model': 'bartl_1.483_fp16', 'Score': 0.46, '%Analysis': 46.0, '%Factoid': 36.0, '%Unanswerable': 6.0, '%Vague': 12.0}, {'QG Model': 'bartl_1.504_fp16', 'Score': 0.39, '%Analysis': 39.0, '%Factoid': 45.0, '%Unanswerable': 9.0, '%Vague': 7.0}]


100%|██████████| 100/100 [14:32<00:00,  8.72s/it]


[{'QG Model': 'bartl_1.483_fp16', 'Score': 0.69, '%Analysis': 69.0, '%Factoid': 16.0, '%Unanswerable': 11.0, '%Vague': 4.0}, {'QG Model': 'bartl_1.504_fp16', 'Score': 0.64, '%Analysis': 64.0, '%Factoid': 18.0, '%Unanswerable': 11.0, '%Vague': 7.0}]


100%|██████████| 100/100 [15:13<00:00,  9.13s/it]


[{'QG Model': 'bartl_1.483_fp16', 'Score': 0.58, '%Analysis': 58.0, '%Factoid': 37.0, '%Unanswerable': 2.0, '%Vague': 3.0}, {'QG Model': 'bartl_1.504_fp16', 'Score': 0.66, '%Analysis': 66.0, '%Factoid': 26.0, '%Unanswerable': 2.0, '%Vague': 6.0}]


100%|██████████| 100/100 [14:55<00:00,  8.96s/it]

[{'QG Model': 'bartl_1.483_fp16', 'Score': 0.13, '%Analysis': 13.0, '%Factoid': 75.0, '%Unanswerable': 6.0, '%Vague': 6.0}, {'QG Model': 'bartl_1.504_fp16', 'Score': 0.11, '%Analysis': 11.0, '%Factoid': 72.0, '%Unanswerable': 7.0, '%Vague': 10.0}]


In [ ]:
# bart_inqui:      43%, 65%, 42%, 13% -- 40.8%
# t5l_dqg:         49%, 64%, 65%, 14% -- 48%
# bartl_dqg_1.509: 34%, 73%  68%, 17% -- 48%
# bartl_dqg_1.510: 35%, 68%, 56%, 08% -- 41.75
# bartl_dqg_1.433: 41%, 69%, 63%, 15% -- 47.5%
# bartl_dqg_1.442: 43%, 64%, 60%, 14% -- 45.25
# bartl_dqg_1.456: 35%, 66%, 70%, 14% -- 46.25
# bartl_dqg_1.483: 48%, 69%, 58%, 13% -- 47%
# bartl_dqg_1.504: 39%, 64%, 66%, 11% -- 45%